In [1]:
# from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms import a2c, ppo, sac
from ray.rllib.algorithms.qmix import QMixConfig
from ray.rllib.algorithms.dqn.dqn import DQNConfig
import gymnasium as gym
from ray.rllib.algorithms.ppo import PPOConfig


config = DQNConfig()

replay_config = config.replay_buffer_config.update( 
    {
        "capacity": 60000,
        "prioritized_replay_alpha": 0.5,
        "prioritized_replay_beta": 0.5,
        "prioritized_replay_eps": 3e-6,
    }
)
config = config.training(replay_buffer_config=replay_config, )

# "MountainCarContinuous-v0"
# "CartPole-v1"
# "Taxi-v3"
env_name = "MountainCarContinuous-v0"

config = (  # 1. Configure the algorithm,
    PPOConfig()
    .environment(env_name)
    .rollouts(num_rollout_workers=2)
    .framework("torch")
    .training(model={"fcnet_hiddens": [256, 256], })
)
# config = (  # 1. Configure the algorithm,
#     ppo.PPOConfig()
#     .environment("Taxi-v3")
#     .rollouts(num_rollout_workers=2)
#     .framework("torch")
#     .training(model={"fcnet_hiddens": [64, 64]})
# )

algo = config.build()  # 2. build the algorithm,

for i in range(100):
    print(f"Iter {i}")
    print(algo.train())  # 3. train it,

    env = gym.make(env_name)
    iter_rewards = []
    for eval in range(2):
        obs, _ = env.reset()
        done = False
        rewards = 0
        while not done:
            action = algo.compute_single_action(obs)
            obs, reward, truncated, terminated, info = env.step(action)
            rewards += reward
            done = truncated or terminated
        iter_rewards.append(rewards)
    print(iter_rewards)

algo.stop()

2023-05-12 16:44:56,305	INFO worker.py:1625 -- Started a local Ray instance.
2023-05-12 16:45:04,707	INFO trainable.py:172 -- Trainable.setup took 10.630 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


Iter 0
{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 13.099694399533654, 'cur_kl_coeff': 0.20000000000000004, 'cur_lr': 5.0000000000000016e-05, 'total_loss': 4.0933211358864945, 'policy_loss': 0.027216548773069536, 'vf_loss': 4.06539794835312, 'vf_explained_var': -0.5195994337399801, 'kl': 0.0035332285408159842, 'entropy': 1.3752437263406734, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 465.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 4000, 'num_env_steps_trained': 4000, 'num_agent_steps_sampled': 4000, 'num_agent_steps_trained': 4000}, 'sampler_results': {'episode_reward_max': -50.132003407152375, 'episode_reward_min': -52.03229032690909, 'episode_reward_mean': -51.02402137788772, 'episode_len_mean': 999.0, 'episode_media': {}, 'episodes_this_iter': 4, 'policy_reward_min': {}, 'pol

(raylet) [2023-05-12 17:27:56,334 E 324084 324084] (raylet) node_manager.cc:3071: 2 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: d6015f7ba840896b5f5028e532af2cc3dcdef90324e68072151e10d4, IP: 172.31.19.228) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip 172.31.19.228`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.


: 

In [1]:
from ray.rllib.algorithms.dqn.dqn import DQNConfig
from ray.rllib.algorithms.ppo import PPOConfig

from ray import air
from ray import tune
config = PPOConfig()
config = config.training( 
    lr=tune.grid_search([0.01, 0.001, 0.0001]))
config = config.environment(env="Taxi-v3") 
tune.Tuner(  
    "PPO",
    run_config=air.RunConfig(stop={"episode_reward_mean":200, "time_total_s": 60*5}),
    param_space=config.to_dict(),
).fit()

2023-05-02 18:26:49,793	INFO worker.py:1625 -- Started a local Ray instance.
2023-05-02 18:26:51,068	INFO tune.py:218 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.


(PPO pid=166174) 2023-05-02 18:26:56,049	WARNING algorithm_config.py:635 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(PPO pid=166174) 2023-05-02 18:26:56,240	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_trained,num_env_steps_trained_this_iter,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_total,training_iteration,trial_id
PPO_Taxi-v3_e8171_00000,72000,"{'ObsPreprocessorConnector_ms': 0.02079153060913086, 'StateBufferConnector_ms': 0.005154132843017578, 'ViewRequirementAgentConnector_ms': 0.12171745300292969}","{'num_env_steps_sampled': 72000, 'num_env_steps_trained': 72000, 'num_agent_steps_sampled': 72000, 'num_agent_steps_trained': 72000}",{},2023-05-02_18-32-12,True,200,{},-200,-1953.2,-2000,20,361,ip-172-31-19-228,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.07239754063995574, 'cur_kl_coeff': 2.4327438354492186, 'cur_lr': 0.01, 'total_loss': 10.024202752882434, 'policy_loss': 0.014961827430193142, 'vf_loss': 10.0, 'vf_explained_var': 0.00020756260041267642, 'kl': 0.003798581422491929, 'entropy': 0.004875025347121446, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 16275.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 72000, 'num_env_steps_trained': 72000, 'num_agent_steps_sampled': 72000, 'num_agent_steps_trained': 72000}",18,172.31.19.228,72000,72000,72000,4000,72000,4000,0,2,0,0,4000,"{'cpu_util_percent': 32.80833333333333, 'ram_util_percent': 25.066666666666666, 'gpu_util_percent0': 0.0, 'vram_util_percent0': 0.00013020833333333333}",166174,{},{},{},"{'mean_raw_obs_processing_ms': 0.3320977958082964, 'mean_inference_ms': 1.0442401992349455, 'mean_action_processing_ms': 0.1398312134925334, 'mean_env_wait_ms': 0.08867559308918423, 'mean_env_render_ms': 0.0}","{'episode_reward_max': -200.0, 'episode_reward_min': -2000.0, 'episode_reward_mean': -1953.2, 'episode_len_mean': 200.0, 'episode_media': {}, 'episodes_this_iter': 20, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [-2000.0, -2000.0, -2000.0, -1991.0, -2000.0, -2000.0, -1991.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -1991.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -1991.0, -2000.0, -1550.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -1973.0, -2000.0, -1982.0, -2000.0, -2000.0, -2000.0, -1991.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -1991.0, -2000.0, -2000.0, -2000.0, -2000.0, -200.0, -2000.0, -2000.0, -200.0, -2000.0, -2000.0, -2000.0, -2000.0, -1991.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -1991.0, -2000.0, -1991.0, -2000.0, -1991.0, -1514.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0, -1991.0, -2000.0, -2000.0, -2000.0, -2000.0, -2000.0], 'episode_lengths': [200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]}, 'sampler_perf': {'mean_

(PPO pid=171453) 2023-05-02 18:32:18,666	WARNING algorithm_config.py:635 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(PPO pid=171453) 2023-05-02 18:32:18,857	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(PPO pid=176756) 2023-05-02 18:37:44,721	WARNING algorithm_config.py:635 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
(PPO pid=176756) 2023-05-02 18:37:44,910	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2023-05-02 18:43:04,042	INFO tune.py:945 -- Total run time: 972.97 seconds (972.51 seconds for the tuning loop).


ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.07239754063995574, 'cur_kl_coeff': 2.4327438354492186, 'cur_lr': 0.01, 'total_loss': 10.024202752882434, 'policy_loss': 0.014961827430193142, 'vf_loss': 10.0, 'vf_explained_var': 0.00020756260041267642, 'kl': 0.003798581422491929, 'entropy': 0.004875025347121446, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 128.0, 'num_grad_updates_lifetime': 16275.5, 'diff_num_grad_updates_vs_sampler_policy': 464.5}}, 'num_env_steps_sampled': 72000, 'num_env_steps_trained': 72000, 'num_agent_steps_sampled': 72000, 'num_agent_steps_trained': 72000}, 'sampler_results': {'episode_reward_max': -200.0, 'episode_reward_min': -2000.0, 'episode_reward_mean': -1953.2, 'episode_len_mean': 200.0, 'episode_media': {}, 'episodes_this_iter': 20, 'policy_reward_min': {}, 'policy_reward_max': {}, 'poli